# 案例: 多智能体系统 (MAS) 仿真

本案例演示了一个纯粹的多智能体、事件驱动的仿真。所有组件之间完全解耦，仅通过消息总线(`MessageBus`)进行通信。

场景包含一个`DigitalTwinAgent`（数字孪生智能体）发布水库状态，以及一个`LocalControlAgent`（本地控制智能体）订阅该状态，并使用PID控制器来操作一个水闸。

### 1. 导入库并加载配置

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent

CONFIG_PATH = 'examples/mas_simulation.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 定义并运行仿真

In [ ]:
def run_mas_simulation(cfg):
    print("\n--- 搭建多智能体仿真示例 ---")
    
    # 1. 仿真和通信设置
    harness = SimulationHarness(config=cfg['simulation'])
    message_bus = harness.message_bus
    topics = cfg['messaging_topics']
    
    # 2. 创建物理组件
    res_cfg = cfg['components']['reservoir']
    reservoir = Reservoir("reservoir_1", res_cfg['initial_state'], res_cfg['params'])
    
    gate_cfg = cfg['components']['gate']
    gate = Gate("gate_1", gate_cfg['initial_state'], gate_cfg['params'], message_bus, topics['gate_action'])
    
    # 3. 创建智能体
    twin_agent = DigitalTwinAgent("twin_agent_reservoir_1", reservoir, message_bus, topics['reservoir_state'])
    
    pid_cfg = cfg['agents']['pid_controller']
    pid_controller = PIDController(setpoint=pid_cfg['setpoint'], Kp=pid_cfg['Kp'], Ki=pid_cfg['Ki'], Kd=pid_cfg['Kd'], min_output=0.0, max_output=gate_cfg['params']['max_opening'])
    
    control_agent = LocalControlAgent(
        "control_agent_gate_1", pid_controller, message_bus,
        observation_topic=topics['reservoir_state'], observation_key='water_level',
        action_topic=topics['gate_action'], dt=cfg['simulation']['dt']
    )
    
    # 4. 添加到仿真平台
    harness.add_component(reservoir)
    harness.add_component(gate)
    harness.add_agent(twin_agent)
    harness.add_agent(control_agent)
    
    # 5. 定义拓扑并运行
    harness.add_connection("reservoir_1", "gate_1")
    harness.build()
    harness.run_mas_simulation()
    
    print("\n--- 仿真结束后的最终状态 ---")
    final_states = {cid: comp.get_state() for cid, comp in harness.components.items()}
    for cid, state in final_states.items():
        state_str = ", ".join(f"{k}={v:.2f}" for k, v in state.items())
        print(f"  {cid}: {state_str}")
        
    return harness.history

history = run_mas_simulation(config)

### 3. 可视化结果

In [ ]:
def plot_history(simulation_history):
    if not simulation_history:
        print("没有历史数据可供绘制。")
        return
        
    results_df = pd.DataFrame([step['reservoir_1'] for step in simulation_history if 'reservoir_1' in step])
    results_df['time'] = [step['time'] for step in simulation_history if 'reservoir_1' in step]
    results_df = results_df.set_index('time')
    
    gate_df = pd.DataFrame([step['gate_1'] for step in simulation_history if 'gate_1' in step])
    gate_df['time'] = [step['time'] for step in simulation_history if 'gate_1' in step]
    gate_df = gate_df.set_index('time')
    results_df['gate_opening'] = gate_df['opening']
    
    fig, ax1 = plt.subplots(figsize=(15, 7))
    fig.suptitle('多智能体仿真结果', fontsize=16)
    
    ax1.plot(results_df.index, results_df['water_level'], 'b-', label='水库水位 (m)')
    ax1.axhline(y=config['agents']['pid_controller']['setpoint'], color='r', linestyle=':', label='目标水位')
    ax1.set_xlabel('时间 (s)')
    ax1.set_ylabel('水位 (m)', color='b')
    ax1.tick_params(axis='y', labelcolor='b')
    ax1.grid(True)
    
    ax2 = ax1.twinx()
    ax2.plot(results_df.index, results_df['gate_opening'], 'g-', label='水闸开度 (m)')
    ax2.set_ylabel('开度 (m)', color='g')
    ax2.tick_params(axis='y', labelcolor='g')
    
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper right')
    
    plt.show()

plot_history(history)